### Build a list with unit dictionaries including below data
- {
- 'musicname':'original filename',
- 'staff_id':int,
- 'start':int,
- 'end':int,
- 'tempo':int,
- 'timesig':'4/4' or '6/8' or...,
- 'beats':'1/4,0; 1/2,1; 1/8,1; 1/8,0' (假設為4/4拍，左為一單位的拍子，順序為：四分休止符、二分音符、八分音符、八分休止符，1/4 + 1/2 + 1/8 + 1/8 = 1)
- }

#### 以下兩格獨立處理{'tempo':int, 'start':int, 'end':int, 'musicname':'original filename'}

In [31]:
from bs4 import BeautifulSoup as bs
import os

#Determine tempo, start measure & end measure
def getTempoMeasure(dirPath, filename, mainList):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs(f.read(), 'xml')
    
    #Select measures of staff#1 because tempos only appear in staff#1.
    staff1Measures = mscx.select('Score > Staff:nth-of-type(1) > Measure')
    
    #Get the 1st tempo.
    tempo = staff1Measures[0].select('Tempo > text')[0].text.rsplit(' ', 1)[-1]
    
    #Set tempos, measure numbers, & filename to the list.
    filename = filename.rsplit('.', 1)[0]
    print filename
    tmp = []
    for i, measure in enumerate(staff1Measures[:-1], 1):
        for t in measure.select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if i % 4 == 0:
            if tmp:
                tempo = max(tmp)
            mainList.append(genDataDic(tempo, i-3, i, filename))
            tmp[:] = []
    
    #Handle last measure different situations with for-else.
    else:
        i += 1
        for t in staff1Measures[-1].select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if tmp:
            tempo = max(tmp)    
        if i % 4 == 1: 
            mainList.append(genDataDic(tempo, i, i, filename))
        elif i % 4 == 2:
            mainList.append(genDataDic(tempo, i-1, i, filename))
        elif i % 4 == 3:
            mainList.append(genDataDic(tempo, i-2, i, filename))
        else:
            mainList.append(genDataDic(tempo, i-3, i, filename))

#Generate unit-data dictionary containing below properties
def genDataDic(tempo, start, end, filename):
    return {'tempo':tempo, 'start':start, 'end':end, 'musicname':filename}

In [32]:
#Determine tempo, start measure, end measure & input the filename
mainList = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getTempoMeasure(dirPath, filename, mainList)
for e in mainList:
    print e

ABBA_Dancing_Queen_CHT
{'start': 1, 'end': 4, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 5, 'end': 8, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 9, 'end': 12, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 13, 'end': 16, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 17, 'end': 20, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 21, 'end': 24, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 25, 'end': 28, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 29, 'end': 32, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 33, 'end': 36, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 37, 'end': 40, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 41, 'end': 44, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 45, 'end': 48, 'tempo': 100, 'musicname': 'ABBA_Dancing_Queen_CHT'}
{'start': 49, 'end': 52, 'tempo': 100, 'musicn

#### 以下n格獨立處理{'staff_id':int, 'timesig':'4/4', 'beats':'1/4,0; 1/2,1; 1/8,1; 1/8,0'}

In [52]:
from bs4 import BeautifulSoup as bs
from decimal import *
import os

#Beat dictionary.
beatDic = {
           'measure':None,
           'whole':Decimal(1),
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }

#Generate unit-data dictionary containing below properties.
def genDataDic2(staff_id, timeSig, beatStr):
    beatStr = beatStr[:-1]
    return {'staff_id':staff_id, 'timesig':timeSig, 'beats':beatStr}

#Called by function doLastMeasure() to simply the process
#of determination if Time Signature changed.
def diffTimeSigsOrNot(mm, diffTimeSigs, timeSig, beatStr):
    if diffTimeSigs == False:
        mainList2.append(genDataDic2(mm, timeSig, beatStr))
    else:
        print 'Time Signature changed, omit this unit.'
        print '------------------------------------'    

#Called by function getBeat() to simplify the process handing last measure.
def doLastMeasure(mm, i, diffTimeSigs, sigN, sigD, beatStr):
    timeSig = str(sigN)+'/'+str(sigD)
    if i % 4 == 1:
        diffTimeSigsOrNot(mm, diffTimeSigs, timeSig, beatStr)
    elif i % 4 == 2:
        diffTimeSigsOrNot(mm, diffTimeSigs, timeSig, beatStr)
    elif i % 4 == 3:
        diffTimeSigsOrNot(mm, diffTimeSigs, timeSig, beatStr)
    else:
        diffTimeSigsOrNot(mm, diffTimeSigs, timeSig, beatStr)

In [53]:
#Called by function countBeat() to determine if last note is a Rest.
#whose beat should be merged with this Rest.
def lastRestOrNot(beatStr, beatSum, key, multi):

    #If beatStr is True, it's not the first note(Rest) of a unit.
    if beatStr:
        
        #If last note is a Rest.
        if beatStr[-2] == '0':
            beatSum += beatDic[key] * Decimal(multi)
            previousBeat = Decimal(beatStr.rsplit(';', 2)[-2].split(',')[0])
            currentBeat = beatDic[key] * Decimal(multi)
            combineBeat = str(previousBeat + currentBeat)
            
            #beatStr contains equivalent to or more than 2 pairs.
            if len(beatStr.rsplit(';', 2)) > 2:
                beatStr = beatStr.rsplit(';', 2)[-3] + ';' + combineBeat + ',0;'
            
            #beatStr contains exactly 1 pair.
            elif len(beatStr.rsplit(';', 2)) == 2:
                beatStr = combineBeat + ',0;'
        
        #If last note is not a Rest.
        else:
            beatSum += beatDic[key] * Decimal(multi)
            beatStr += str(beatDic[key] * Decimal(multi))+',0;'
            
    #If beatStr is False, it's the first note(Rest) of a unit.
    else:
        beatSum += beatDic[key] * Decimal(multi)
        beatStr += str(beatDic[key] * Decimal(multi))+',0;'
    return beatStr, beatSum

#Called by function countBeat() to handle Chord tags in a measure
def ifChord(beatStr, beatSum, note):
    print note.name
    key = note.find('durationType').text
    if note.find('dots'):
        dots = int(note.find('dots').text)
        if dots == 1:
            beatSum += beatDic[key] * Decimal(1.5)
            beatStr += str(beatDic[key] * Decimal(1.5))+',1;'
        elif dots == 2:
            beatSum += beatDic[key] * Decimal(1.75)
            beatStr += str(beatDic[key] * Decimal(1.75))+',1;'
        elif dots == 3:
            beatSum += beatDic[key] * Decimal(1.875)
            beatStr += str(beatDic[key] * Decimal(1.875))+',1;'
    else:
        beatSum += beatDic[key]
        beatStr += str(beatDic[key])+',1;'
    return beatStr, beatSum

#Called by function countBeat() to handle Rest tags in a measure
def ifRest(beatStr, beatSum, note):
    print note.name
    key = note.find('durationType').text
    if note.find('dots'):
        dots = int(note.find('dots').text)
        if dots == 1:
            beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.5)
        elif dots == 2:
            beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.75)
        elif dots == 3:
            beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1.875)
    else:
        beatStr, beatSum = lastRestOrNot(beatStr, beatSum, key, 1)
    return beatStr, beatSum

#Called by function getBeat()
#to count beats & accumulate the beat-string of a unit
def countBeat(measure, beatStr):
    beatSum = 0
    for note in measure.children:
        if str(type(note)) == '<class \'bs4.element.Tag\'>':
            if note.name == 'Chord' and not note.find('track'):
                beatStr, beatSum = ifChord(beatStr, beatSum, note)
            elif note.name == 'Rest' and not note.find('track'):
                beatStr, beatSum = ifRest(beatStr, beatSum, note)
    print beatSum
    if beatSum == beatDic['measure']:
        print True, ', beatSum: ' + str(beatSum)
    else:
        print False, ', beatSum: ' + str(beatSum)
    print beatStr
    print '------------------------------------'
    return beatStr

In [54]:
#Determine time signature & beats
#Note: About <tick><track> tags indicating at least
#1 additional melody in a staff, omit them at first.
def getBeat(dirPath, filename, mmList, mainList2):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs(f.read(), 'xml')
    
    #Get 1st time signature
    staff1TimeSig = mscx.select_one('Score > Staff:nth-of-type(1) TimeSig')
    sigN = int(staff1TimeSig.find('sigN').text)
    sigD = int(staff1TimeSig.find('sigD').text)
    beatDic['measure'] = Decimal(sigN)/sigD
    diffTimeSigs = False
    
    #Set staff_id, time signature & beats to the list
    for mm in mmList:
        beatStr = ''
        for i, measure in enumerate(mscx.select('Score > \
                              Staff:nth-of-type('+str(mm)+') > \
                              Measure')[:-1], 1):
            print '[Staff #'+str(mm)+', Measure #'+str(i)+']'
            
            #Determine Time Signature & detect if it changes
            if measure.find('TimeSig'):
                sigN = int(measure.find('sigN').text)
                sigD = int(measure.find('sigD').text)
                if Decimal(sigN)/sigD != beatDic['measure']:
                    beatDic['measure'] = Decimal(sigN)/sigD
                    if i % 4 == 2 or i % 4 == 3:
                        diffTimeSigs = True
                        continue
                    elif i % 4 == 0:
                        diffTimeSigs = False
                        beatStr = ''
                        print 'Time Signature changed, omit this unit.'
                        print '------------------------------------'
                        continue
                        
            #Count beats & accumulate the beat-string of a unit
            beatStr = countBeat(measure, beatStr)
            
            #Generate a unit dictionary per 4 measures
            if i % 4 == 0:
                if diffTimeSigs == True:
                    diffTimeSigs = False
                    beatStr = ''
                    print 'Time Signature changed, omit this unit.'
                    print '------------------------------------'
                    continue
                timeSig = str(sigN)+'/'+str(sigD)
                mainList2.append(genDataDic2(mm, timeSig, beatStr))
                beatStr = ''
                
        #Handle last-measure situations with for-ELSE.
        else:
            i += 1
            lastMeasure = mscx.select('Score > \
                                       Staff:nth-of-type('+str(mm)+') > \
                                       Measure')[-1]
            print '[Staff #'+str(mm)+', Measure #'+str(i)+']'
            if not lastMeasure.find('TimeSig'):
                beatStr = countBeat(lastMeasure, beatStr)
                doLastMeasure(mm, i, diffTimeSigs, sigN, sigD, beatStr)
            else:
                sigN = int(lastMeasure.find('sigN').text)
                sigD = int(lastMeasure.find('sigD').text)
                if Decimal(sigN)/sigD == beatDic['measure']:
                    beatStr = countBeat(lastMeasure, beatStr)
                    doLastMeasure(mm, i, diffTimeSigs, sigN, sigD, beatStr)
                else:
                    print 'Time Signature changed, omit this unit.'
                    print '------------------------------------'

In [55]:
#Numbers of staffs which play main melodies
mmList = [1]

#Determine tempo, start measure, end measure & input the filename
#Determine time signature, beats, staff-d,
mainList2 = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getBeat(dirPath, filename, mmList, mainList2)
for j, e in enumerate(mainList2, 1):
    print '[Unit #'+str(j)+']'
    print e
    print '------------------------------------'

[Staff #1, Measure #1]
Rest
1
True , beatSum: 1
1,0;
------------------------------------
[Staff #1, Measure #2]
Rest
1
True , beatSum: 1
2,0;
------------------------------------
[Staff #1, Measure #3]
Rest
1
True , beatSum: 1
3,0;
------------------------------------
[Staff #1, Measure #4]
Chord
Chord
Rest
Chord
Rest
Chord
Chord
Chord
Chord
1.0000
True , beatSum: 1.0000
3,0;0.1875,1;0.0625,1;0.125,0;0.125,1;0.0625,0;0.0625,1;0.0625,1;0.0625,1;0.25,1;
------------------------------------
[Staff #1, Measure #5]
Chord
Chord
Rest
Chord
Chord
Chord
Chord
Chord
Chord
1.0000
True , beatSum: 1.0000
0.1875,1;0.0625,1;0.125,0;0.125,1;0.125,1;0.0625,1;0.0625,1;0.125,1;0.125,1;
------------------------------------
[Staff #1, Measure #6]
Chord
Chord
Rest
Chord
Rest
Chord
Chord
Chord
Chord
1.0000
True , beatSum: 1.0000
0.1875,1;0.0625,1;0.125,0;0.125,1;0.125,1;0.0625,1;0.0625,1;0.125,1;0.125,1;0.1875,1;0.0625,1;0.125,0;0.125,1;0.0625,0;0.0625,1;0.0625,1;0.0625,1;0.25,1;
---------------------------

In [ ]:
for i in range(0,len(music.select('Chord'))):
    if len(music.select('Chord')[i].select('Tie')):
        key1 = music.select('Chord')[i].select('durationType')[0].text
        key2 = music.select('Chord')[i+1].select('durationType')[0].text
        beatSum = beatDic[key1]+beatDic[key2]
        print 'tie id = '+ music.select('Chord')[i].select('Tie')[0].get('id')
        print beatSum